In [1]:
%matplotlib inline

# Plotting setup
import matplotlib.pyplot as plt
import geopandas

# Package imports
import numpy as np
import pandas as pd

# Allow import of own scripts #
import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
###############################

# Own functions
from src.load_taxi_data import load_taxi_data
from src.taxi_zones_loader import taxi_zones_loader

In [11]:
'''
Load taxi data for all taxi types for January 2019 before Covid.
This will take a while because total data amount is about 2 GB!
'''
# df_taxi_2019_01 = load_taxi_data(['yellow', 'green', 'fhv'], [2019], [1])

Will download... yellow_tripdata_2019-01.csv
Will download... green_tripdata_2019-01.csv
Will download... fhv_tripdata_2019-01.csv


In [12]:
'''
Save data to csv to save time.
This will take a while as total file size is about 2 GB!
'''
# df_taxi_2019_01.to_csv('../dat/df_taxi_2019_01.csv', encoding='utf-8')

In [34]:
# Read previously saved dataframe from csv file
df_taxi_2019_01 = pd.read_csv('../dat/df_taxi_2019_01.csv')

**Definition:** The **most frequent route** between any two locations _A_ and _B_ is the unidirectional route _A*_ &rarr; _B*_ with the highest trip frequency.

In [3]:
df_taxi_2019_01

,Unnamed: 0,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,fleet
0,0,2019-01-01 00:46:40,2019-01-01 00:53:20,151.0,239.0,yellow
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,239.0,246.0,yellow
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,236.0,236.0,yellow
3,3,2018-11-28 15:52:25,2018-11-28 15:55:45,193.0,193.0,yellow
4,4,2018-11-28 15:56:57,2018-11-28 15:58:33,193.0,193.0,yellow
...,...,...,...,...,...,...
31429515,23130805,2019-01-31 23:21:00,2019-01-31 23:28:52,NaN,265.0,fhv
31429516,23130806,2019-01-31 23:38:04,2019-01-31 23:45:43,NaN,265.0,fhv
31429517,23130807,2019-01-31 23:53:58,2019-02-01 00:25:04,NaN,265.0,fhv
31429518,23130808,2019-01-31 23:54:53,2019-02-01 00:00:14,NaN,265.0,fhv


**Preprocessing:**

In [35]:
# Remove unnecessary column to save space
if 'Unnamed: 0' in df_taxi_2019_01.columns:
    df_taxi_2019_01.drop(columns='Unnamed: 0', inplace=True) # inplace=True modifies the dataframe itself

In [36]:
# Get names of indexes for which column LocationID has value 0 => This is an undefined location!
indexNames = df_taxi_2019_01[ (df_taxi_2019_01['PULocationID'] == 0) | (df_taxi_2019_01['DOLocationID'] == 0) ].index

# Delete these row indexes from taxi trip data
df_taxi_2019_01.drop(indexNames , inplace=True)

In [37]:
print('There are {} rows with undefined location (i.e., DOLocationID = 0 or PULocationID = 0).'.format(df_taxi_2019_01[df_taxi_2019_01['DOLocationID'] == 0].shape[0] + df_taxi_2019_01[df_taxi_2019_01['PULocationID'] == 0].shape[0]))

There are 0 rows with undefined location (i.e., DOLocationID = 0 or PULocationID = 0).


In [38]:
# Identify all non-numeric values and convert them to NaN
df_taxi_2019_01['PULocationID'] = pd.to_numeric(df_taxi_2019_01['PULocationID'], errors='coerce')
df_taxi_2019_01['DOLocationID'] = pd.to_numeric(df_taxi_2019_01['DOLocationID'], errors='coerce')

In [39]:
# Replace infinite values with the NaN values
df_taxi_2019_01.replace([np.inf, -np.inf], np.nan, inplace=True)

In [40]:
# Drop all rows with NaN
df_taxi_2019_01.dropna(axis=0, inplace=True)

In [41]:
# Some locations are stored as float but we need discrete int values for later grouping
df_taxi_2019_01['PULocationID'] = df_taxi_2019_01['PULocationID'].astype(int)
df_taxi_2019_01['DOLocationID'] = df_taxi_2019_01['DOLocationID'].astype(int)

In [42]:
# Test: Are any NaN left?
print('There are {} rows with NaN values left.'.format(df_taxi_2019_01[df_taxi_2019_01['PULocationID'].isnull()].shape[0]))

There are 0 rows with NaN values left.


In [43]:
print('About {:.2f}% of the entire data could not be used due to missing information (NaN).'.format(100*(1-df_taxi_2019_01.shape[0]/31429520)))

About 5.81% of the entire data could not be used due to missing information (NaN).


**Remove row from the wrong year:**

In [13]:
df_taxi_2019_01

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,fleet
0,2019-01-01 00:46:40,2019-01-01 00:53:20,151,239,yellow
1,2019-01-01 00:59:47,2019-01-01 01:18:59,239,246,yellow
2,2018-12-21 13:48:30,2018-12-21 13:52:40,236,236,yellow
3,2018-11-28 15:52:25,2018-11-28 15:55:45,193,193,yellow
4,2018-11-28 15:56:57,2018-11-28 15:58:33,193,193,yellow
...,...,...,...,...,...
31429479,2019-01-31 23:32:45,2019-01-31 23:32:49,247,45,fhv
31429481,2019-01-31 23:37:00,2019-01-31 23:54:00,265,265,fhv
31429482,2019-01-31 23:58:00,2019-02-01 00:59:00,265,265,fhv
31429483,2019-01-31 23:46:00,2019-02-01 00:21:00,265,265,fhv


In [44]:
# Remove wrong year form pick-ups and rop-offs.
# The or "|" is necessary to include fringe cases on new year's!
df_taxi_2019_01 = df_taxi_2019_01[df_taxi_2019_01['pickup_datetime'].str.contains('2019') | df_taxi_2019_01['dropoff_datetime'].str.contains('2019')]

In [45]:
# Quick test for wrong year
df_taxi_2019_01[df_taxi_2019_01['pickup_datetime'].str.contains('2018')]

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,fleet
112,2018-12-31 17:22:55,2019-01-01 16:57:23,132,181,yellow
497,2018-12-31 23:56:19,2019-01-01 00:53:44,48,265,yellow
505,2018-12-31 23:56:54,2019-01-01 00:01:55,170,170,yellow
557,2018-12-31 23:56:57,2019-01-01 00:30:30,239,158,yellow
878,2018-12-31 23:58:36,2019-01-01 00:04:18,229,229,yellow
...,...,...,...,...,...
7668960,2018-12-31 23:59:00,2019-01-01 00:06:13,130,131,green
7669365,2018-12-31 16:03:20,2019-01-01 15:13:23,41,74,green
7669939,2018-12-31 23:58:49,2019-01-01 00:03:02,41,41,green
7673443,2018-12-31 16:13:01,2019-01-01 15:23:54,247,116,green


**Remove routes with IDs that refer to locations outside the city:**

In [17]:
# IDs 264 and 265 refer to locations outside the city => Rows containing them need to be dropped
taxi_zones_loader()

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights
...,...,...,...
260,261,Manhattan,World Trade Center
261,262,Manhattan,Yorkville East
262,263,Manhattan,Yorkville West
263,264,Unknown,NV


In [52]:
idx_outside = [264, 265]

df_taxi_2019_01 = df_taxi_2019_01[(df_taxi_2019_01['PULocationID'] != idx_outside[0]) & (df_taxi_2019_01['PULocationID'] != idx_outside[0]) & \
                                  (df_taxi_2019_01['DOLocationID'] != idx_outside[1]) & (df_taxi_2019_01['DOLocationID'] != idx_outside[1])]

In [50]:
# Quick test for wrong IDs
df_taxi_2019_01[ (df_taxi_2019_01['DOLocationID'] == 264) ]

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,fleet


In [51]:
df_taxi_2019_01

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,fleet
0,2019-01-01 00:46:40,2019-01-01 00:53:20,151,239,yellow
1,2019-01-01 00:59:47,2019-01-01 01:18:59,239,246,yellow
7,2019-01-01 00:21:28,2019-01-01 00:28:37,163,229,yellow
8,2019-01-01 00:32:01,2019-01-01 00:45:39,229,7,yellow
9,2019-01-01 00:57:32,2019-01-01 01:09:32,141,234,yellow
...,...,...,...,...,...
31429423,2019-01-31 23:14:37,2019-01-31 23:21:25,243,244,fhv
31429424,2019-01-31 23:20:41,2019-01-31 23:53:05,61,10,fhv
31429425,2019-01-31 23:33:45,2019-01-31 23:45:16,163,75,fhv
31429426,2019-01-31 23:57:42,2019-02-01 00:23:08,161,256,fhv


**Most popular routes:**

In [35]:
# Gather metadata
print('PULocationID: Min = {}, Max = {}'.format(df_taxi_2019_01['PULocationID'].min(), df_taxi_2019_01['PULocationID'].max()))
print('DOLocationID: Min = {}, Max = {}'.format(df_taxi_2019_01['DOLocationID'].min(), df_taxi_2019_01['DOLocationID'].max()))

PULocationID: Min = 1, Max = 265
DOLocationID: Min = 1, Max = 265


In [31]:
by_puid = df_taxi_2019_01.groupby('PULocationID')

In [32]:
for puid, frame in by_puid:
    print(f"First 2 entries for {puid!r}")
    print("------------------------")
    print(frame.head(2), end="\n\n")

First 2 entries for 1
------------------------
           pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
33961  2019-01-01 02:19:27  2019-01-01 02:19:32             1             1   
60640  2019-01-01 05:52:45  2019-01-01 05:52:53             1             1   

        fleet  
33961  yellow  
60640  yellow  

First 2 entries for 2
------------------------
            pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
627596  2019-01-04 06:55:07  2019-01-04 08:10:35             2            65   
741757  2019-01-04 16:05:29  2019-01-04 16:51:32             2           211   

         fleet  
627596  yellow  
741757  yellow  

First 2 entries for 3
------------------------
           pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
78181  2019-01-01 10:05:06  2019-01-01 10:12:23             3            51   
78740  2019-01-01 10:06:36  2019-01-01 10:42:47             3            61   

        fleet  
78181  yellow  
78740  ye

            pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
23555   2019-01-01 01:46:48  2019-01-01 02:06:55            64            92   
105264  2019-01-01 13:05:20  2019-01-01 13:41:22            64           220   

         fleet  
23555   yellow  
105264  yellow  

First 2 entries for 65
------------------------
          pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
1198  2018-12-31 23:59:47  2019-01-01 00:13:02            65            79   
1248  2019-01-01 00:41:13  2019-01-01 01:06:10            65           141   

       fleet  
1198  yellow  
1248  yellow  

First 2 entries for 66
------------------------
          pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
1059  2019-01-01 00:54:27  2019-01-01 01:06:07            66            25   
4393  2019-01-01 00:39:57  2019-01-01 01:00:14            66           164   

       fleet  
1059  yellow  
4393  yellow  

First 2 entries for 67
------------------------
 

             pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
8321582  2019-01-01 00:30:19  2019-01-01 00:35:53           110           109   
8404016  2019-01-01 01:02:04  2019-01-01 01:16:44           110            84   

        fleet  
8321582   fhv  
8404016   fhv  

First 2 entries for 111
------------------------
            pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
178911  2019-01-01 22:09:24  2019-01-01 22:09:29           111           111   
226216  2019-01-02 09:22:00  2019-01-02 09:30:04           111           181   

         fleet  
178911  yellow  
226216  yellow  

First 2 entries for 112
------------------------
          pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
2016  2019-01-01 00:53:10  2019-01-01 01:09:03           112            36   
2067  2019-01-01 00:56:14  2019-01-01 01:17:08           112           234   

       fleet  
2016  yellow  
2067  yellow  

First 2 entries for 113
-----------

27  yellow  

First 2 entries for 162
------------------------
        pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
45  2019-01-01 00:46:44  2019-01-01 00:56:18           162           140   
67  2019-01-01 00:38:28  2019-01-01 00:53:39           162           151   

     fleet  
45  yellow  
67  yellow  

First 2 entries for 163
------------------------
        pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
7   2019-01-01 00:21:28  2019-01-01 00:28:37           163           229   
12  2019-01-01 00:45:21  2019-01-01 01:31:05           163            25   

     fleet  
7   yellow  
12  yellow  

First 2 entries for 164
------------------------
         pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
90   2019-01-01 00:50:22  2019-01-01 01:08:19           164           262   
175  2019-01-01 00:40:33  2019-01-01 00:44:25           164           170   

      fleet  
90   yellow  
175  yellow  

First 2 entries for 165
--

745  yellow  

First 2 entries for 210
------------------------
           pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
20260  2019-01-01 01:11:47  2019-01-01 01:37:48           210            11   
76021  2019-01-01 11:00:34  2019-01-01 11:07:49           210           155   

        fleet  
20260  yellow  
76021  yellow  

First 2 entries for 211
------------------------
         pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
178  2019-01-01 00:25:03  2019-01-01 01:02:07           211            48   
185  2019-01-01 00:12:48  2019-01-01 00:23:48           211           232   

      fleet  
178  yellow  
185  yellow  

First 2 entries for 212
------------------------
           pickup_datetime     dropoff_datetime  PULocationID  DOLocationID  \
21425  2019-01-01 01:07:43  2019-01-01 02:06:48           212           115   
21494  2019-01-01 01:47:57  2019-01-01 02:05:08           212            41   

        fleet  
21425  yellow  
21494 

In [40]:
# Count UNIQUE rows
by_puid.get_group(2)['DOLocationID'].value_counts()

117    4
2      3
265    3
124    3
263    2
132    2
65     1
48     1
230    1
50     1
256    1
30     1
209    1
239    1
145    1
211    1
68     1
82     1
10     1
141    1
234    1
244    1
107    1
138    1
Name: DOLocationID, dtype: int64

In [41]:
by_puid.get_group(2)

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,fleet
627596,2019-01-04 06:55:07,2019-01-04 08:10:35,2,65,yellow
741757,2019-01-04 16:05:29,2019-01-04 16:51:32,2,211,yellow
1244788,2019-01-06 18:14:08,2019-01-06 18:51:09,2,107,yellow
2616923,2019-01-12 05:06:24,2019-01-12 05:19:14,2,117,yellow
4471019,2019-01-19 11:59:57,2019-01-19 12:28:15,2,2,yellow
5588055,2019-01-24 08:25:00,2019-01-24 09:00:52,2,244,yellow
6326082,2019-01-26 19:36:15,2019-01-26 19:41:23,2,263,yellow
6992591,2019-01-29 16:56:11,2019-01-29 17:04:58,2,234,yellow
7324358,2019-01-30 20:31:27,2019-01-30 20:40:50,2,141,yellow
7401392,2019-01-31 06:26:03,2019-01-31 06:33:25,2,2,yellow


In [44]:
puid, frame = next(iter(by_puid))  # First tuple from iterator

In [45]:
puid

1

In [50]:
frame.head(3)

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,fleet
33961,2019-01-01 02:19:27,2019-01-01 02:19:32,1,1,yellow
60640,2019-01-01 05:52:45,2019-01-01 05:52:53,1,1,yellow
63320,2019-01-01 06:22:24,2019-01-01 06:23:23,1,1,yellow


In [56]:
frame["DOLocationID"].value_counts()  # Count for PULocationID == 1

265    503
1      457
231     87
239     82
166     78
      ... 
155      1
86       1
44       1
64       1
5        1
Name: DOLocationID, Length: 142, dtype: int64

How to interpret the results?
<pre>
DOID   Count               DOID: DOLocationID = Drop-off location when pick-up location is 1.
-------------
265    503
1      457
231     87
239     82
166     78
      ... 
155      1
86       1
44       1
64       1
5        1
</pre>

In [72]:
'''
 * Count how often each possible destination (DOLocationID) was approached for all possible origins (PULocationID).
 * Series.reset_index can be used with name= set to create a DataFrame instead of a Series and give a name to the
   otherwise unnamed values column that will hold the result of value_counts().
'''
df_trip_counts = df_taxi_2019_01.groupby("PULocationID")["DOLocationID"].value_counts().reset_index(name='trip_count')

In [75]:
# Result is a pandas series object
trip_counts = df_taxi_2019_01.groupby("PULocationID")["DOLocationID"].value_counts()

In [76]:
trip_counts

PULocationID  DOLocationID
1             265             503
              1               457
              231              87
              239              82
              166              78
                             ... 
265           195               1
              202               1
              207               1
              227               1
              253               1
Name: DOLocationID, Length: 59494, dtype: int64

In [74]:
# In contrast to the series above, let's have a look at the corresponding DataFrame
df_trip_counts

,PULocationID,DOLocationID,trip_count
0,1,265,503
1,1,1,457
2,1,231,87
3,1,239,82
4,1,166,78
...,...,...,...
59489,265,195,1
59490,265,202,1
59491,265,207,1
59492,265,227,1


``trip_count`` stores the number of trips from ``PULocationID`` to ``DOLocationID``.

In [92]:
df_trip_counts.sort_values(by=['trip_count'], ascending=False)

,PULocationID,DOLocationID,trip_count
59016,264,264,914029
28474,132,265,68268
52713,237,236,62729
52457,236,236,55722
52458,236,237,54814
...,...,...,...
40869,184,90,1
40868,184,89,1
40867,184,88,1
40866,184,79,1


In [106]:
df_zones_info = taxi_zones_loader()
df_zones_info.drop(columns='service_zone', inplace=True)

In [107]:
df_zones_info

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights
...,...,...,...
260,261,Manhattan,World Trade Center
261,262,Manhattan,Yorkville East
262,263,Manhattan,Yorkville West
263,264,Unknown,NV


In [104]:
df_trip_counts

,PULocationID,DOLocationID,trip_count
0,1,265,503
1,1,1,457
2,1,231,87
3,1,239,82
4,1,166,78
...,...,...,...
59489,265,195,1
59490,265,202,1
59491,265,207,1
59492,265,227,1


In [113]:
# Find zone and burough names for DOLocationID
df_trip_counts = df_trip_counts.merge(left_on='DOLocationID', right=df_zones_info, right_on='LocationID')

In [114]:
df_trip_counts

,PULocationID,DOLocationID,trip_count,LocationID,Borough,Zone
0,1,265,503,265,Unknown,NaN
1,2,265,3,265,Unknown,NaN
2,3,265,1537,265,Unknown,NaN
3,4,265,1071,265,Unknown,NaN
4,5,265,92,265,Unknown,NaN
...,...,...,...,...,...,...
59489,129,199,1,199,Bronx,Rikers Island
59490,135,199,1,199,Bronx,Rikers Island
59491,170,199,1,199,Bronx,Rikers Island
59492,252,199,1,199,Bronx,Rikers Island


In [115]:
# Column not needed anymore
df_trip_counts.drop(columns='LocationID', inplace=True)

In [119]:
df_trip_counts.rename(columns={"Borough": "DOBorough", "Zone": "DOZone"}, inplace=True)

In [120]:
df_trip_counts

,PULocationID,DOLocationID,trip_count,DOBorough,DOZone
0,1,265,503,Unknown,NaN
1,2,265,3,Unknown,NaN
2,3,265,1537,Unknown,NaN
3,4,265,1071,Unknown,NaN
4,5,265,92,Unknown,NaN
...,...,...,...,...,...
59489,129,199,1,Bronx,Rikers Island
59490,135,199,1,Bronx,Rikers Island
59491,170,199,1,Bronx,Rikers Island
59492,252,199,1,Bronx,Rikers Island


In [121]:
# Find zone and burough names for PULocationID
df_trip_counts = df_trip_counts.merge(left_on='PULocationID', right=df_zones_info, right_on='LocationID')

In [123]:
# Column not needed anymore
df_trip_counts.drop(columns='LocationID', inplace=True)

# Rename columns to make clear that they belong to the pick-ups
df_trip_counts.rename(columns={"Borough": "PUBorough", "Zone": "PUZone"}, inplace=True)

Table containing trip counts from ``PULocationID`` &rarr; ``DOLocationID``:

In [125]:
df_trip_counts.sort_values(by=['trip_count'], ascending=False)

,PULocationID,DOLocationID,trip_count,DOBorough,DOZone,PUBorough,PUZone
59112,264,264,914029,Unknown,NV,Unknown,NV
28459,132,265,68268,Unknown,NaN,Queens,JFK Airport
52717,237,236,62729,Manhattan,Upper East Side North,Manhattan,Upper East Side South
52461,236,236,55722,Manhattan,Upper East Side North,Manhattan,Upper East Side North
52471,236,237,54814,Manhattan,Upper East Side South,Manhattan,Upper East Side North
...,...,...,...,...,...,...,...
26753,124,245,1,Staten Island,West Brighton,Queens,Howard Beach
26762,124,3,1,Bronx,Allerton/Pelham Gardens,Queens,Howard Beach
26782,124,220,1,Bronx,Spuyten Duyvil/Kingsbridge,Queens,Howard Beach
26785,124,159,1,Bronx,Melrose South,Queens,Howard Beach
